# Weather Data Analysis

### Import Dependencies

In [1]:
#import dependencies

import random
import requests
import pandas as pd
import csv
import matplotlib.pyplot as plt

from citipy import citipy
from config import api_key


### Random latitudes

- Range Latitudes: -90 to +90
- Range Longitudes: -180 to +180

In [5]:
coord_list = []

#Generate 600 random lats and lons

ix = 0
jx = 200
#jx = 2

while(ix<18):
    
    range1 = -90 + (ix*10)
    range2 = range1 + 10 
    
    range3 = -180 + (ix*20)
    range4 = range3 + 20 
    
    #print(f'#### {ix}) Get {jx} random Lats from {range1}, {range2} and random Long from {range3}, {range4}')
    
    for i in range(jx):
        
        rand_lat = round( random.uniform(range1, range2) , 5) 
        rand_lon = round( random.uniform(range3, range4) , 5) 

        tmp = (rand_lat,rand_lon)
        coord_list.append(tmp)
    
    ix += 1

print(coord_list[0])
## I had issues finding UNIQUE cities because the rando was too wide.. 
## so to solve this I started getting data from ranges from 10 to 10.. this is getting random numbers 
# from -90 to -80, then from -80 to -70

(-84.86794, -175.78546)


### Get the cities

These have to be at least 500 unique cities

In [6]:
cities = []

for coord in coord_list:
    
    city = citipy.nearest_city(coord[0],coord[1])
    city_name = city.city_name
    country_code = city.country_code
    #print(f'processing #  City Name: {city_name} - Country Code: {country_code}')
    
    tmp = (city_name, country_code)
    cities.append(tmp)

In [7]:
len(cities)

3600

In [8]:
unique_cities = list(set(cities))

In [9]:
len(unique_cities)

693

### Data Retrival

OpenWeatherMap - Weather API

In [10]:
# Save config information.
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"

# Build partial query URL
query_url_tmp = f'{url}appid={api_key}&units={units}&'


In [11]:
#coord_list = [(lat,lon)]
city_name_list = []
country_code_lsit = []

lon_api_list = []
lat_api_list = []
temp_list = []
humidity_list = []
wind_list = []
clouds = []
country_api = []
city_id_api = []
city_api = []

i = 0
# Loop through the list of cities and perform a request for data on each
for cities in unique_cities:
    
    i += 1
    print(f'[#######] {i}: City: {cities[0].capitalize()}')
    
    query_url = query_url_tmp + f'q={cities[0]},{cities[1]}'
    
    #print(query_url) #do not print when pushing to github
    try:
        response = requests.get(query_url).json()
        response['sys']['country']
    except:
        print(f'[#######] CITY NOT FOUND: {cities[0].capitalize()}')
        continue
    
    city_name_list.append(cities[0])
    country_code_lsit.append(cities[1])
    
    lon_api_list.append(response['coord']['lon'])
    lat_api_list.append(response['coord']['lat'])
    temp_list.append(response['main']['temp'])
    humidity_list.append(response['main']['humidity'])
    wind_list.append(response['wind']['speed'])
    clouds.append(response['clouds']['all'])
    country_api.append(response['sys']['country'])
    city_id_api.append(response['id'])
    city_api.append(response['name'])
    
    #print(response)



[#######] 1: City: Ayolas
[#######] 2: City: Gondar
[#######] CITY NOT FOUND: Gondar
[#######] 3: City: Bouar
[#######] 4: City: Reconquista
[#######] 5: City: Shihezi
[#######] 6: City: Doha
[#######] 7: City: Yenagoa
[#######] 8: City: Chaman
[#######] 9: City: Srednekolymsk
[#######] CITY NOT FOUND: Srednekolymsk
[#######] 10: City: Mitzic
[#######] 11: City: Kitob
[#######] 12: City: Sangmelima
[#######] CITY NOT FOUND: Sangmelima
[#######] 13: City: Sharan
[#######] 14: City: Brodowski
[#######] CITY NOT FOUND: Brodowski
[#######] 15: City: Jiayuguan
[#######] 16: City: Armacao dos buzios
[#######] CITY NOT FOUND: Armacao dos buzios
[#######] 17: City: Ortigueira
[#######] 18: City: Arbagar
[#######] 19: City: Riaba
[#######] CITY NOT FOUND: Riaba
[#######] 20: City: Buraydah
[#######] 21: City: Eskasem
[#######] CITY NOT FOUND: Eskasem
[#######] 22: City: Khormuj
[#######] CITY NOT FOUND: Khormuj
[#######] 23: City: Mendoza
[#######] 24: City: Nola
[#######] 25: City: Berdigestya

[#######] CITY NOT FOUND: Gardan diwal
[#######] 238: City: Capao da canoa
[#######] 239: City: Najran
[#######] 240: City: Belaya gora
[#######] 241: City: Bitam
[#######] 242: City: Gusinoye ozero
[#######] 243: City: Curanilahue
[#######] 244: City: Ribeira do pombal
[#######] 245: City: Fontem
[#######] 246: City: Pervomayskiy
[#######] CITY NOT FOUND: Pervomayskiy
[#######] 247: City: Ancud
[#######] 248: City: Terra roxa
[#######] 249: City: La rioja
[#######] 250: City: Glubokoe
[#######] CITY NOT FOUND: Glubokoe
[#######] 251: City: Kyabe
[#######] 252: City: Faya
[#######] CITY NOT FOUND: Faya
[#######] 253: City: Baghdad
[#######] CITY NOT FOUND: Baghdad
[#######] 254: City: Betare oya
[#######] CITY NOT FOUND: Betare oya
[#######] 255: City: Neuquen
[#######] 256: City: Santiago
[#######] 257: City: Kichera
[#######] 258: City: Ouesso
[#######] 259: City: Korem
[#######] 260: City: Mbandaka
[#######] 261: City: Abonnema
[#######] 262: City: Amaigbo
[#######] 263: City: Taman

[#######] CITY NOT FOUND: Sinkat
[#######] 468: City: Mecca
[#######] 469: City: Canavieiras
[#######] 470: City: Warri
[#######] 471: City: Sao joaquim
[#######] 472: City: Ziarat
[#######] 473: City: Itatiba
[#######] 474: City: Aksha
[#######] 475: City: Barguzin
[#######] 476: City: Abohar
[#######] 477: City: Denau
[#######] CITY NOT FOUND: Denau
[#######] 478: City: Florianopolis
[#######] 479: City: Babanusah
[#######] CITY NOT FOUND: Babanusah
[#######] 480: City: Pirajui
[#######] 481: City: Olavarria
[#######] 482: City: Bayanday
[#######] 483: City: Amambai
[#######] 484: City: Loum
[#######] 485: City: Sao francisco de assis
[#######] CITY NOT FOUND: Sao francisco de assis
[#######] 486: City: Churapcha
[#######] 487: City: Debre sina
[#######] 488: City: Sao joao da barra
[#######] 489: City: Conde
[#######] 490: City: Doka
[#######] 491: City: Doha
[#######] CITY NOT FOUND: Doha
[#######] 492: City: Mataura
[#######] CITY NOT FOUND: Mataura
[#######] 493: City: Sao miguel

In [12]:
# Cities found on the WeatherAPI

len(city_api)

592

### Pandas Dataframe creation

In [13]:
weather_data = pd.DataFrame(
                {'City_citipy': city_name_list,
                'Country_code_citipy': country_code_lsit,
                'City_ID': city_id_api,
                'City': city_api,
                'Country': country_api,
                'Longitude': lon_api_list,
                'Latitude': lat_api_list,
                'Temperature': temp_list,
                'Humidity': humidity_list,
                'Wind': wind_list,
                'Clouds': clouds
                })


weather_data.head()

,City_citipy,Country_code_citipy,City_ID,City,Country,Longitude,Latitude,Temperature,Humidity,Wind,Clouds
0,ayolas,py,3439378,Ayolas,PY,-56.84,-27.39,26.34,79,5.01,64
1,bouar,cf,2387926,Bouar,CF,15.59,5.94,18.34,45,1.86,0
2,reconquista,ar,3429594,Reconquista,AR,-59.64,-29.14,21.69,99,2.71,92
3,shihezi,cn,1529195,Shihezi,CN,86.03,44.30,-18.24,97,1.31,44
4,doha,qa,290030,Doha,QA,51.53,25.29,19.86,56,6.70,0


In [14]:
weather_data.count()

City_citipy            592
Country_code_citipy    592
City_ID                592
City                   592
Country                592
Longitude              592
Latitude               592
Temperature            592
Humidity               592
Wind                   592
Clouds                 592
dtype: int64

---

### Data analysis

- Temperature (F) vs. Latitude
- Humidity (%) vs. Latitude
- Cloudiness (%) vs. Latitude
- Wind Speed (mph) vs. Latitude